In [1]:
from pandas import DataFrame
from pyarrow import Table
from pyarrow.flight import connect, FlightDescriptor
from random import random

client = connect("grpc://0.0.0.0:8080")

In [2]:
for i in client.list_flights():
    print(i)

<pyarrow.flight.FlightInfo schema=x: double
y: double descriptor=<pyarrow.flight.FlightDescriptor path=[b'linear_regression']> endpoints=[<pyarrow.flight.FlightEndpoint ticket=<pyarrow.flight.Ticket ticket=b'linear_regression'> locations=[<pyarrow.flight.Location b'grpc://0.0.0.0:8080'>]>] total_records=-1 total_bytes=-1>


In [22]:
data = DataFrame([
    {"x": i*1.0, "y": i*.5+.5}
    for i in range(100000)
])

table = Table.from_pandas(data)

writer, reader = client.do_exchange(FlightDescriptor.for_path("linear_regression"))

writer.begin(table.schema)
writer.write_table(table)
writer.done_writing()


data = reader.read_pandas()

data

,x,predict
0,0.0,0.5
1,100.0,50.5
2,200.0,100.5
3,300.0,150.5
4,400.0,200.5
5,500.0,250.5
6,600.0,300.5
7,700.0,350.5
8,800.0,400.5
9,900.0,450.5
